In [ ]:
%pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=0b617ea442a67dbbc6bcdb3107c9a00356d6545ad9df7ee80ef6356014379e37
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.volatility import BollingerBands

plt.style.use('ggplot')


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from sklearn.metrics import accuracy_score  # ✅ THIS FIXES THE ERROR
from google.colab import files
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving BAJFINANCE.csv to BAJFINANCE (1).csv
Saving BHARTIARTL.csv to BHARTIARTL (1).csv
Saving HDFCBANK.csv to HDFCBANK (1).csv
Saving INFY.csv to INFY (1).csv
Saving ITC.csv to ITC (1).csv
Saving LT.csv to LT (1).csv
Saving RELIANCE.csv to RELIANCE (1).csv
Saving SUNPHARMA.csv to SUNPHARMA (1).csv
Saving TATACHEM.csv to TATACHEM (1).csv
Saving TATAMOTORS.csv to TATAMOTORS (1).csv


In [ ]:
stock_files = {
    'HDFC Bank': 'HDFCBANK.csv',
    'Reliance': 'RELIANCE.csv',
    'Infosys': 'INFY.csv',
    'ITC': 'ITC.csv',
    'Tata Motors': 'TATAMOTORS.csv',
    'Sun Pharma': 'SUNPHARMA.csv',
    'L&T': 'LT.csv',
    'Bharti Airtel': 'BHARTIARTL.csv',
    'Bajaj Finance': 'BAJFINANCE.csv',
    'Tata Chemicals': 'TATACHEM.csv'
}


In [ ]:
bollinger_results = []

for stock_name, filename in stock_files.items():
    try:
        df = pd.read_csv(filename)
        df.columns = [col.lower() for col in df.columns]  # ensure lowercase
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.set_index('date', inplace=True)
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df.dropna(subset=['close'], inplace=True)

        # Calculate Bollinger Bands
        bb = BollingerBands(close=df['close'], window=20, window_dev=2)
        df['bb_mavg'] = bb.bollinger_mavg()
        df['bb_upper'] = bb.bollinger_hband()
        df['bb_lower'] = bb.bollinger_lband()

        # Signal: Buy when price crosses above lower band, sell when crosses below upper band
        df['signal'] = 0
        df.loc[(df['close'] < df['bb_lower']) & (df['close'].shift(1) >= df['bb_lower'].shift(1)), 'signal'] = 1
        df.loc[(df['close'] > df['bb_upper']) & (df['close'].shift(1) <= df['bb_upper'].shift(1)), 'signal'] = -1

        # Backtest
        df['return'] = df['close'].pct_change()
        df['strategy_return'] = df['signal'].shift(1) * df['return']
        df.dropna(inplace=True)

        # Accuracy
        actual_direction = (df['return'] > 0).astype(int)
        predicted_direction = (df['signal'].shift(1) > 0).astype(int)
        predicted_direction = predicted_direction[df.index.isin(actual_direction.index)]
        accuracy = accuracy_score(actual_direction, predicted_direction)

        # Sharpe Ratio
        sharpe = (df['strategy_return'].mean() / df['strategy_return'].std()) * np.sqrt(252) if df['strategy_return'].std() != 0 else 0

        # Max Drawdown
        cum_returns = (1 + df['strategy_return']).cumprod()
        peak = cum_returns.cummax()
        drawdown = (peak - cum_returns) / peak
        max_dd = drawdown.max()

        # Store results
        bollinger_results.append({
            'Stock': stock_name,
            'Accuracy': round(accuracy, 2),
            'Sharpe Ratio': round(sharpe, 2),
            'Max Drawdown (%)': round(max_dd * 100, 2)
        })

        print(f"✅ {stock_name} Bollinger strategy completed.")

    except Exception as e:
        print(f"❌ {stock_name} failed: {e}")


✅ HDFC Bank Bollinger strategy completed.
✅ Reliance Bollinger strategy completed.
✅ Infosys Bollinger strategy completed.
✅ ITC Bollinger strategy completed.
✅ Tata Motors Bollinger strategy completed.
✅ Sun Pharma Bollinger strategy completed.
✅ L&T Bollinger strategy completed.
✅ Bharti Airtel Bollinger strategy completed.
✅ Bajaj Finance Bollinger strategy completed.
✅ Tata Chemicals Bollinger strategy completed.


In [ ]:
# Convert to DataFrame
bollinger_df = pd.DataFrame(bollinger_results)

# Save to CSV
bollinger_df.to_csv("BollingerBands_Performance.csv", index=False)

# Download in Colab
files.download("BollingerBands_Performance.csv")

# Show summary
bollinger_df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Stock,Accuracy,Sharpe Ratio,Max Drawdown (%)
0,HDFC Bank,0.48,0.08,13.45
1,Reliance,0.48,-0.58,49.49
2,Infosys,0.48,-0.09,26.22
3,ITC,0.50,0.14,18.84
4,Tata Motors,0.50,-0.31,52.69
5,Sun Pharma,0.49,-0.02,32.55
6,L&T,0.50,-0.09,31.20
7,Bharti Airtel,0.50,0.36,17.15
8,Bajaj Finance,0.48,-0.20,32.93
9,Tata Chemicals,0.48,0.52,16.42
